# 「文字列情報の利用」と「検証用データを用いたモデルの評価」
前回は以下のことを学びました．
 - pandasによるcsvの読み込みと簡単な操作（列・行へのアクセス，`numpy.ndarray`への変換）
 - sklearnの基本的な使い方
 - 提出までの流れ：
   1. 特徴ベクトルの作成
   2. モデル・アルゴリズムの選定，学習
   3. 予測
   4. 提出

その一方で，以下のことは**行いませんでした**：
 - 文字列情報の利用：文字列で表現されている情報はどうやって用いるのか？
 - 予測の投稿前の定量的な評価

## 今回学ぶこと

 - 文字列情報のone-hotエンコーディング
 - 検証用データを用いたモデルの投稿前の評価
 - 付録：欠損値処理

## （復習）予測モデル構築の流れ

機械学習を用いて予測モデルを構築し，未知の（テスト）データに対して予測を行う手順は，主に以下のようになります．
1. データを用意し，特徴ベクトルを作る
2. どのような手法（モデル）を使うかを決める
3. モデルを学習する方法を決め，学習する
4. 未知のデータに予測を行う（そして本演習では提出する）


## （復習）データ読み込み
前回と同様に，まずはデータの読み込みを行います．
データの読み込みはpandasの`read_csv`で行えます．
ライブラリを使うためには`import`する必要があります．
詳細は前回の資料を参照してください．
今回も前回と同じロジスティック回帰のモデルを使うため，`LogisticRegression`を`import`します．

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.linear_model import LogisticRegression #ロジスティック回帰LogisticRegressionを動かす．

Colabの場合は次のセルを動かし，更にその次のセルのパスを適宜変更してください．

In [ ]:
# Google Colab を用いている場合，このセルを実行
from google.colab import drive
drive.mount('/content/drive') # google driveをマウント（＝Colabから使えるようにする）

In [ ]:
# Google Colab を用いている場合，このセルを実行
d_train = pd.read_csv("drive/My Drive/data/train.csv") # 訓練データを読み込む．TFがGoogle Driveの一番上にdataディレクトリを置いた場合はこのようなパスになった
d_test = pd.read_csv("drive/My Drive/data/test.csv") # テストデータを読み込む．TFがGoogle Driveの一番上にdataディレクトリを置いた場合はこのようなパスになった

In [ ]:
# 自身のPython環境で動かしている場合，このセルを実行（パスは適宜変更）
#d_train = pd.read_csv("data/train.csv") # 訓練データを読み込む
#d_test = pd.read_csv("data/test.csv") # テストデータを読み込む

前回と同じように，読み込んだデータを表示して確認します．

In [ ]:
print("訓練データ")
print(d_train)
print("\nテストデータ")
print(d_test)

前回と同様にいくつかの変数を用意します．
 - `n_train`, `n_test`：訓練データ数，テストデータ数.
 - `y_train`：訓練データの目標値．`pop`メソッドによって元のデータフレームから取り除いて作る．

In [ ]:
print("訓練データとテストデータの数を取得")
n_train = len(d_train)
n_test = len(d_test)
print(f"訓練データ数：{n_train}，テストデータ数：{n_test}")
print("\n 出力情報を取り出す．strokeの列が消えている")
# targetの値
y_train = d_train.pop('stroke')
y_train = y_train.values # numpyのarrayに変換
print(d_train)
print(y_train)

## （復習）数値情報の特徴ベクトルを作る

またしても復習ですが，**まず**前回と同じ特徴ベクトルを作ります．
入力の情報として，以下が与えられています：

 - id：整数値．1人ごとに割り当てられる固有の数値．
 - gender：文字列．性別を意味し，値は"Male"，"Female"，"Other"のいずれか．
 - age：整数値．年齢．
 - hypertension：整数値．高血圧患者なら1，でなければ0．
 - heart_disease：整数値．心臓病患者なら1，でなければ0．
 - ever_married：文字列．結婚経験があるかを意味し，値は"No"か"Yes"のいずれか．
 - work_type：文字列．労働形態を意味し，値は"children"，"Govt_jov"，"Never_worked"，"Private"，"Self-employed"のいずれか．
 - Residence_type：文字列．居住地のタイプを意味し，値は"Rural"か"Urban"のいずれか．
 - avg_glucose_level：実数値．平均血糖値．
 - bmi：実数値．ボディマス指数(Body Mass Index)．
 - smoking_status：文字列．喫煙習慣を意味し，値は"formerly smoked"，"never smoked"，"smokes"，"Unknown"のいずれか．
 - stroke：**今回予測する値**．整数値．脳卒中患者なら1，でなければ0．

この中で，"gender"，"ever_married"，"work_type"，"Residence_type"，"smoking_status"が文字列（string）の情報です．前回はこれらを用いずに，"id"，"age"，"hypertension"，"heat_disease"，"avg_glucose_level"の元から数値的な情報である5つだけを用いました．pandasではブラケット（角括弧）`[]`を用いることで特定の列や行を取り出すことができ，また`.values`で配列の中身だけを取り出せるのでした． したがって，以下のようにすることで数値の情報だけを用いた特徴ベクトルの行列を作ることができます．

In [ ]:
columns_num = ["id","age","hypertension","heart_disease","avg_glucose_level"]
X_train_num = d_train[columns_num].values
X_test_num = d_test[columns_num].values

sklearnを用いる基本的な手順は，
1. モデルのインスタンスを作成
2. 作成したモデルオブジェクトを
**`fit`メソッド**
を用いて学習．`fit`メソッドには訓練データの入力と目標値（つまり，行列とベクトル）を渡す．
3. 学習したモデルを用いて
**`predict`メソッド**
で予測．`predict`メソッドにはデータの入力（つまり行列）を渡す．

でした．これは，以下のようにして実行できます．

また，ロジスティック回帰では分類の確率も出すことができます．`predict_proba`メソッドを用いるとデータ毎の[予測ラベルが0の確率, 予測ラベルが1の確率]が分かります．

In [ ]:
# 手順1：LogisticRegressionのインスタンスの作成
lr = LogisticRegression()
# 手順2：上で作ったオブジェクトの学習
lr.fit(X_train_num, y_train)
# 手順3：テストデータに対する予測
y_pred_test_lr_proba = lr.predict_proba(X_test_num)[:,1] # 予測ラベルが1の確率を出力
print(y_pred_test_lr_proba)
np.savetxt(X=y_pred_test_lr_proba, fname='y_pred_lr_proba.txt')

さて，この予測のスコアはどうでしょうか？

参考までに，私の環境では

y_pred_lr.txt：0.500000

y_pred_lr_proba.txt：0.465234

となりました．これを見ると，前回実装したロジスティック回帰モデルは，（性能指標としてAUC-ROCを用いた場合）あまり良い性能とは言えなさそうです
（環境により多少スコアが変化するかもしれません）．
原因を探るため，データセットの中をもう少し見てみましょう．
試しに，訓練データ内のstrokeの値の分布を見てみます．
データの集計方法には色々ありますが，今回は`numpy`の`unique()`メソッドを使ってみましょう．

In [ ]:
nums, counts = np.unique(y_train, return_counts=True) # y_trainの中の各値の個数を集計
# nums:y_train中に含まれる値の(重複しない)リスト
# counts:y_trainに含まれる，対応するnumsの要素の個数
print(nums)
print(counts)

この訓練データにはstrokeが0のデータが3398件，1のデータが179件あることが分かりました．訓練データ全体では3577件のデータがあるので，strokeが0のデータが訓練データ全体の約95%を占めていることになります．すなわち，（訓練データの正解ラベルに関して）**データの分布に大きな偏りがある**ことになります（このようなデータセットは不均衡データセットやlabel-imbalanced datasetなどと呼ばれることがあります）．このとき，学習にどのような影響があるでしょうか？

前回，目的関数として交差エントロピー誤差関数（cross-entropy error function）を使うと仮定しました．交差エントロピー誤差関数は，以下のように定義されていたのでした．

$$
  E(\mathbf{w}) = - \sum_{n=1}^N \{ t_n \ln y_n + (1-t_n) \ln(1-y_n) \} 
$$

ただし，$y_n = \sigma ( \mathbf{w}^{\top} {\mathbf{x}_n} ) $($n$番目のデータに対する$y$の予測値)，$t_n \in \{0,1\}$($n$番目のデータの正解クラス)です．

この式を変形すると，以下のようになります．

$$
  \begin{split} 
    E(\mathbf{w}) &= - \left( \sum_{n:t_n = 0} \{ t_n \ln y_n + (1-t_n) \ln(1-y_n) \} + \sum_{n:t_n=1} \{ t_n \ln y_n + (1-t_n) \ln(1-y_n) \} \right) \\
    &= - \sum_{n:t_n = 0} \ln(1-y_n) - \sum_{n:t_n=1} \ln y_n
  \end{split}
$$

ここで，$n:t_n=0(1)$とは$t_n=0(1)$であるような$n$（の集合），すなわち正解ラベルが$0(1)$であるデータを意味するものとします．

すると$t_n=0$であるデータの数が$t_n=1$であるデータの数と比べて十分に大きいとき，上の式の第1項は第2項よりも目的関数に大きな影響を与えます．つまり，$t_n=0$であるデータの数が$t_n=1$であるデータの数と比べて「多すぎる」とき，この目的関数は **$t_n=0$であるデータを過剰に，$t_n=1$であるデータを過少に評価してしまう**恐れがあります．

[scikit-learnのロジスティック回帰のページ](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)を見てみるとclass_weightというハイパーパラメータ（実行前に設定する変数）があり，デフォルトでは全てのクラスについて同じ重みが設定されているようです．今の場合，データ数の少ないクラスのデータに大きな重みをかけたいので，class_weight='balanced'に設定するのが良さそうです．早速試してみましょう．

In [ ]:
# 手順1：LogisticRegressionのインスタンスの作成，class_weight='balanced'に設定
lr_balanced = LogisticRegression(class_weight='balanced')
# 手順2：上で作ったオブジェクトの学習
lr_balanced.fit(X_train_num, y_train)
# 手順3：テストデータに対する予測
y_pred_lr_balanced = lr_balanced.predict_proba(X_test_num)[:,1]
print(y_pred_lr_balanced)
np.savetxt(X=y_pred_lr_balanced, fname='y_pred_lr_balanced.txt')

この`y_pred_lr_balanced.txt`を提出してみるとどうなるでしょうか？

私の環境では提出してみると0.733946のスコアが出ました．以前のスコアと比べて大きく改善しています！

このように，思ったようにスコアが出ないときにどの部分を改善できるかを考え仮説を立て，実装して確認するというのがコンペにおける流れとなります．ここからは，さらにスコアを高める方法について考えてみます．

## 文字列情報のone-hotエンコーディング

文字列情報である"gender"，"ever_married"，"work_type"，"Residence_type"，"smoking_status"は，値段の予測に役に立つと考えられるのでやはり利用したいです． 機械学習手法は基本的に数値情報しか用いることができないため，文字列の情報をどうにかして数値に変換する必要があります． 一般に，非数値的な変数のことを
**質的変数**
や**カテゴリカル変数**と呼びます．

一つの方法として，**文字列一つ一つに適当に数字（例えば0（or 1）から始まる整数値）を割り当てる**というのが考えられます． 例えば，"gender"では"Female"を1，"Male"を2に変換する，"work_type"では"children"を1，"Govt_jov"を2，"Never_worked"を3…，といった方法です． しかし，この方法は多くのケースでは適切ではありません． まず，基本的に，多くの予測モデルにおいて**特徴の値は重要な要素**です．
例えば線形モデルは以下の式で与えられます：
$$y(\mathbf{x}; \mathbf{w}) = \sum_{j=1}^D x_jw_j.$$ 
$x_j$というのは$w_j$の係数になっていますから，$x_j$の絶対値が大きい時，$w_j$の予測結果に対する影響度は大きくなります．
しかし，**カテゴリに割り当てた数字の値そのものに（基本的には）意味はない**はずです．
"Never_worked"を1にして，"children"を3にしてもよいはずです．
けれども，予測モデルに与えた時，数字の大きさが考慮されてしまいますから，この方法は適切ではないでしょう（したがって，**与えられたデータが数字で表現されていても，数字の値そのものに意味がない場合はそのまま使うのは適切ではない**ということになります）．

文字列情報の数値的な情報への変換方法で最もメジャーな方法として**one-hotエンコーディング**があります． one-hotエンコーディングでは，**一つのカテゴリカル変数はカテゴリー数の次元のベクトルに変換**されます． 変換されたベクトルは，一つの要素が一つのカテゴリに対応していて，対応する要素の値が1でそれ以外の要素の値が0であるようなベクトルです．

例えば，"work_type"変数は"children"，"Govt_jov"，"Never_worked"，"Private"，"Self-employed"の5種類の値を取ります． この時，これらの文字列はそれぞれ，以下のような5次元のベクトルに変換されます．

 - "children" &rarr; $(1,0,0,0,0)$
 - "Govt_jov" &rarr; $(0,1,0,0,0)$
 - "Never_worked" &rarr; $(0,0,1,0,0)$
 - "Private" &rarr; $(0,0,0,1,0)$
 - "Self-employed" &rarr; $(0,0,0,0,1)$

さて，この変換は適切なのでしょうか？ 質的変数にも色々ありますから，全ての質的変数をダミー変数にすることが適切であるとは限りませんが，少なくとも雑に整数値を割り当てるよりはずっと良いです．
上の例の7次元ベクトルに対する線形モデルを例に考えてみます．
線形モデルの式は$y(\mathbf{x}; \mathbf{w}) = \sum_{j=1}^D x_jw_j$でした．
$\mathbf{x}$は今はどれか一つが$1$でそれ以外全てが$0$であるようなベクトルです．
$x_j=0$の時，$w_j$は使われませんから，$w_j$はjに対応する労働形態に関する目的変数の予測値を表しています．
$w_0$は"children"に関する予測，$w_1$は"Govt_jov"に関する予測…といったようになり，雑に整数値を割り当てるよりはずっと良さそうですね．

one-hotエンコーディングでは，それぞれのカテゴリに一つの要素を割り当てて，完全に異なる特徴として扱います． そのため，例えば日付のような情報が文字列として与えられたとき，「日付が近い場合，値段の傾向も近い」と考えるのが自然かと思いますが，日付が異なれば完全に別のものとして扱うため，そのような傾向を陽に扱うことが難しくなります． ですが，今回はとりあえず全ての文字列情報をone-hotエンコードします．


## pandasによるone-hotエンコーディング
それでは実際に"gender"，"ever_married"，"work_type"，"Residence_type"，"smoking_status"の文字列情報をone-hotエンコーディングしてみましょう！
一見すると「書けるけど確実にそこそこ面倒だな」と思うかもしれません．
Pythonには辞書型という使いやすいハッシュテーブルが提供されていますが，それでも少し面倒かもしれません．
幸運なことに，pandasには`get_dummies`という非常に便利な関数が用意されています．
`get_dummies`を使うと，`DataFrame`の特定の列を簡単にone-hotエンコーディングできます．

ここまで退屈だったと思いますので，今回のQuizです．

### Quiz

[get_dummiesのドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)
を読んで，以下のセルを埋め，訓練データとテストデータの"gender"，"ever_married"，"work_type"，"Residence_type"，"smoking_status"をone-hotエンコードしてください．
**数値情報はone-hotエンコードせず，そのままにすること**．

この時，訓練データとテストデータを別々にエンコードすると，訓練データとテストデータが異なるエンコードをされてしまう可能性があります．
そこで，**訓練データとテストデータを一旦連結**し，**連結したものをエンコード**し，**エンコード後に再び分ける**ことにします．
以下のセルでは最初に訓練データとテストデータを連結させた`d_train_test`を作っています（上が`d_train`，下が`d_test`）．
また，`columns_cat`はカテゴリカル変数の列名を集めたリストです．
これらを利用すると楽でしょう（使わなくともよいですが）．
訓練データとテストデータを分離させる方法については，前回の資料が参考になります．

変換後の行列が**21列になっていれば正しいです**．

In [ ]:
d_train_test = pd.concat([d_train, d_test], axis=0) # 訓練とテストを連結
columns_cat = ["gender","ever_married", "work_type", "Residence_type", "smoking_status"] # カテゴリカル変数の列名

d_train_test_onehot = # ここを埋める．get_dummiesを使ってone-hotエンコーディング
d_train_test_onehot.pop("bmi") # 今回はbmiデータを使わず，捨てる
d_train_onehot = # ここを埋める．d_train_test_onehotの訓練データ部分
d_test_onehot = # ここを埋める．d_train_test_onehotのテストデータ部分
X_train_onehot = d_train_onehot.values # np.arrayに変換
X_test_onehot = d_test_onehot.values  # np.arrayに変換

`d_train_onehot`を`print`してみましょう．
数値情報を全てそのまま使いつつ，列の数が増えていることが分かります．

In [ ]:
print(d_train_onehot)

今までと同様に予測結果を出力してみましょう．ここではテストデータに対する予測結果を`y_pred_lr_balanced.txt`というファイルに格納します．

In [ ]:
lr_balanced.fit(X_train_onehot, y_train)
y_pred_lr_onehot = lr_balanced.predict_proba(X_test_onehot)[:,1]
print(y_pred_lr_onehot)
np.savetxt(X=y_pred_lr_onehot, fname='y_pred_lr_onehot.txt')

この結果も提出してみましょう．

私の環境では，0.838499というスコアが出ました．先程の`y_pred_lr_balanced.txt`の結果よりも向上していますね．今回は新しく作成した特徴ベクトルがうまくスコア向上に働いたようです．

## 検証用データを用いたモデルの評価
### 過学習
さて，これまでの手順でさまざまなモデルを作成し，合計で4つの予測結果を提出しました．しかし，予測手法としては様々なモデルが考えられ，また`LogisticRegression`の`solver`などのようにモデルには様々なハイパーパラメータがあります．全てのモデルを試して最もよい性能のものを探索できれば良いのですが，一日の投稿回数が5回に制限されているため全てのパターンを一日に投稿することはできません．
今後さまざまな予測手法を試すことを考えると，**試行錯誤する要素はどんどん増加するため，毎回の予測結果を全て投稿して評価することは現実的ではありません**．
そこで，**投稿せずに・投稿する前に**モデルを評価することを考えます．
投稿する前にモデルの評価をして，悪そうなモデルの予測結果は投稿せず，良さそうな場合だけ投稿する，というのは妥当な戦略でしょう．
また，コンペに限らず，モデルの事前の評価は機械学習手法の運用において常に非常に重要です．
機械学習を用いたサービスを考えた時に，とりあえず作ってみた予測モデルを本番環境で動かしてみるのは恐ろしいことでしょう．
本番環境で動かす前に一度評価して，良さそうであれば本番環境で動かすべきであるはずです．

さて，それではどのように投稿せずに・投稿する前にモデルを評価すれば良いのでしょうか？
コンペのスコアは**予測と正解**を用いて計算されます（今回はAUC-ROC）．
残念ながら，テストデータの正解はわかっていないので，実際にコンペサイトから返ってくるスコアと同じものを事前に計算することはできません．
そこで，正解がわかっているデータ，すなわち訓練データについて予測を行って，誤差を計算して性能を見積もる，というのが考えられます．
しかし，この方法には問題があります．
モデルは**訓練データの誤差を小さくするように学習**しています．
非常に複雑なモデルを用いた時，訓練データに対する誤差をとにかく小さくしようとして（複雑なため，そのようなことが可能），その結果，訓練データに対して非常に精度の良い予測を行うが，訓練データに含まれないデータに対しては精度の低い予測を行ってしまう，ということがあります．
このような現象・状態を**過学習・過適合（overfitting）**と言います．データサイエンスの初回の講義で登場した**次数の大きい多項式回帰**が過学習の良い例です．
過学習するモデルというのは驚くほど簡単に作れてしまうため，**訓練データに対する誤差を用いてモデルを評価することは不適切です**．

例えば，以下のセルを動かしてみましょう．
以下のセルでは，過学習させるようにハイパーパラメータを選んだ[**カーネルリッジ回帰**](https://scikit-learn.org/stable/modules/generated/sklearn.kernel_ridge.KernelRidge.html)というモデルを動かしています（どのようなモデルかは今は気にしなくて良いです）．


In [ ]:
from sklearn.kernel_ridge import KernelRidge # カーネルリッジ回帰を使えるようにする
kr = KernelRidge(kernel="rbf", gamma=1.0, alpha=0.0001)
kr.fit(X_train_num, y_train)
y_pred_train_kr = kr.predict(X_train_num) # 訓練データに対して予測
y_pred_test_kr = kr.predict(X_test_num) # テストデータに対して予測
np.savetxt(X=y_pred_test_kr, fname="y_pred_kr.txt") # テストデータの結果を保存

では，カーネルリッジ回帰の訓練データに対する予測`y_pred_train_kr`と訓練データの目標値`y_train`の間でAUC-ROCを計算してみます．
自分で実装しても良いですが，sklearnに`roc_auc_score`という名前で既に実装されています．
以下のようにimportして使います．
詳しくは[ドキュメント](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)を読んでみてください．

In [ ]:
from sklearn.metrics import roc_auc_score
auc_train_kr = roc_auc_score(y_train, y_pred_train_kr) # 最初の引数に目標値（正解），次に予測を入れるとスコアが返ってくる
print(f"カーネルリッジ回帰の訓練スコア：{auc_train_kr}")

おそらく，このカーネルリッジ回帰の訓練スコアは1.0程度になると思います．ROU-AUCが1.0となるのはデータを完璧に分類できているときに限るので，これはもう**訓練データに対しては完全に正しく予測している**と言って良いでしょう．しかし，最後に"y_pred_kr.txt"としてファイル出力したテストデータに対する予測結果を実際に投稿してみると，**0.5前後の非常に悪いスコア**となっているはずです（信じがたい方はぜひ投稿してみてください）．

繰り返しになりますが，この例からもわかるように，学習に用いたデータで評価することは適切ではありません．モデル次第では訓練データでの評価がテストデータでの評価をうまく反映する場合もありますが，未知のデータに対する推論性能の評価を行うという目的からすると，（既知である）訓練データによって評価を行うということは妥当とは言い難いでしょう．

### 検証データの作成と検証データを用いた評価

学習に使ったデータでモデルを評価するのは不適切でした．
そこで，正解のわかっているデータの一部を**学習には用いず検証（評価）に用いる**ということを行います．
このようなデータを訓練データ（訓練集合）に対して **検証データ（検証集合，validation data, validation set）** と言います．

では実際にデータの分割をしてみましょう．
まず，数値情報だけを用いた`X_num`について行ってみます．
とりあえずここでは，ラベル付きデータ全体のおよそ**8割を訓練データ，残りを検証用**とします．
特に，現在のラベル付きデータの前半8割を訓練データ，後ろの2割を検証データとします．
先程のQuizでは`DataFrame`の分割を行いましたが，`np.array`の分割も同様に行うことができます．

In [ ]:
n_valid = int(0.2*n_train) # 検証データの数．ラベル付きデータ数n_trainの2割を検証データにする
n_train = n_train - n_valid # 訓練データ数を計算し直す．検証データ以外全てなので，全体（n_train）からn_validationを引く 
X_valid_num = X_train_num[n_train:] # 現在のX_train_numの後半2割 = 前半8割以降
X_train_num = X_train_num[:n_train] # 現在のX_train_numの前半8割．再代入する（変数名を使い回す）

# 目標値も同じように分割する
y_valid = y_train[n_train:]
y_train = y_train[:n_train]

では，ここまでに登場したロジスティック回帰，カーネルリッジ回帰のモデルを，分割した訓練データで学習し，検証データで評価してみましょう．

In [ ]:
# ロジスティック回帰の学習・評価
lr_balanced.fit(X_train_num, y_train)
y_pred_valid_lr = lr_balanced.predict_proba(X_valid_num)[:,1]
auc_lr = roc_auc_score(y_valid, y_pred_valid_lr)
print(f"ロジスティック回帰：{auc_lr}")

# カーネルリッジ回帰の学習・評価
kr.fit(X_train_num, y_train)
y_pred_valid_kr = kr.predict(X_valid_num)
auc_kr = roc_auc_score(y_valid, y_pred_valid_kr)
print(f"カーネルリッジ回帰：{auc_kr}")

結果が出てきました．
カーネルリッジ回帰は訓練データに対する予測性能は良かったですが，訓練に使っていない検証データに対する予測性能が非常に悪い，教科書に載せたいような過学習の例となっていますね．

同様に，今回作ったone-hotベクトルに対しても訓練・検証の分割を行ってみます．

In [ ]:
X_valid_onehot = X_train_onehot[n_train:] # 現在のX_train_onehot_fillの後半2割 = 前半8割以降
X_train_onehot = X_train_onehot[:n_train] # 現在のX_train_onehot_fillの前半8割．再代入する

同じように学習と評価を行います．

In [ ]:
# ロジスティック回帰の学習・評価
lr_balanced.fit(X_train_onehot, y_train)
y_pred_valid_lr_onehot = lr_balanced.predict_proba(X_valid_onehot)[:,1]
auc_lr_onehot = roc_auc_score(y_valid, y_pred_valid_lr_onehot)
print(f"ロジスティック回帰：{auc_lr_onehot}")

# カーネルリッジ回帰の学習・評価
kr.fit(X_train_onehot, y_train)
y_pred_valid_kr_onehot = kr.predict(X_valid_onehot)
auc_kr_onehot = roc_auc_score(y_valid, y_pred_valid_kr_onehot)
print(f"カーネルリッジ回帰：{auc_kr_onehot}")

ロジスティック回帰に比べ，カーネルリッジ回帰のスコアが低くなっています．
ただ，これはカーネルリッジ回帰が悪いのではなく，カーネルリッジ回帰の**使い方**が悪いです．
今後の資料では，使うのがやや難しいモデルをちゃんと使う，ということも行う予定です（カーネルリッジ回帰・ロジスティック回帰以外にも手法は色々あるので，ぜひ自身で調べて使ってみてください）．

今回はとりあえず前半8割を訓練，後半2割を検証としました．
しかし，分割の仕方によって結果も変わってしまいます．
分割・検証とハイパーパラメータの決定のもう少し賢い・便利な方法を次回行います．

## まとめ
 - 質的（カテゴリカル）データを変換する方法としてone-hotエンコーディングがある．一つ一つのカテゴリに特徴を割り当てて，そのカテゴリかそうでないかを0/1で表す．
 - 予測モデルを実際に使う前に（予測を提出する前に）モデルの評価をする必要がある．ラベルのある（もともと訓練用として渡されている）データを，訓練用のデータと検証用のデータに分割し，分割された訓練データだけを用いて学習し，学習に用いなかった検証用のデータを用いてモデルの評価をする．
 
 
量的変数と質的変数（カテゴリカルデータ）について，例えば「名義尺度」「順序尺度」「間隔尺度」等で調べてみると細かい分類やどのような操作が意味を持つのか（持たないのか）が出てくると思うので，興味がある方は調べてみると良いと思います．

## 付録：欠損値処理

機械学習ではさまざまなデータセットが用いられますが，必ずしも完全なデータばかりでなく，そのいくつかの値が欠損している場合があります．例えば数値のデータを扱うとき，データを眺めてみるとところどころに`NaN`という値が含まれていることがあります．この`NaN`（Not a Number）とは，無限大であったり，あるいはゼロ除算（値を0で割ること）で生じたような，数値として表せないデータを意味します．また，例えば他のデータセットで誕生日の情報を文字列として持つ際に，空文字列（""）が含まれるかもしれません．このように，データとして欠けている値を欠損値といい，一般に学習前に何らかの前処理を行う必要があります．データに欠損値が含まれる場合の対処の仕方は様々にあり，

 1. 欠損値を含む列をデータセットから削除する
 2. 欠損値を何らかの値（代表値）で補完する
 3. 欠損値を含む列の値を目的変数とした予測モデルを使う

などがあります．前回と今回のnotebookでは 1. の方法を使っていました． 1. の方法を使った場合，データセットから欠損値を取り除いた部分については完全なデータセットとなっているので欠損値の影響を受けずに済むという利点があります．しかし，例えば1000行あるデータセットの情報Aの列に10個の欠損値があった場合， 1. の方法では10個の欠損値を扱う必要はなくなりますが，欠損していない990個のデータを捨てることになってしまいます．ここでは，付録として 2. の方法について軽く説明します． 2. の方法では，10個の欠損データを加工して取り扱えるようにすることで990個のデータを活かすことを考えます．

突然ですが，クイズです．

問題:
データセットにおいて，とある列のデータが
```Python
[ 96 102 99 101 102 ? 100 ]
```
となっており，`?`の値が欠損しています．このとき`?`の値として最も可能性が高いのは次のうちどれでしょう？

A) 100  
B) 1000  
C) -100 

上の条件だけでは正確な答えは導けませんが，A)100を選んだ人が多いのではないでしょうか．1000や-100よりは100の方が?の値として
合っていそうに見えると思います． このように「合っていそうな(違和感の少ない)」値で欠損値を補完するというのが，今回説明する欠損値処理のイメージです．

補完先の値の候補は色々考えられますが，最もシンプルな方法の一つは，同じ行の欠損していないデータから代表値を求め，それに置き換えるという方法です．代表値は数値の分布の特徴を示す値であり，平均値はメジャーな代表値の一種です．他に，中央値や最頻値も代表値として扱うことができます．ここでは，例として平均値で欠損値を補完する方法を紹介します．

欠損値が含まれるかを確認するには，`isnull()`メソッドを用います．このメソッドにより，あるデータが欠損値であれば`True`，でなければ`False`となります．実際に下のセルで実行してみましょう．

In [ ]:
print(d_train.isnull())

`isnull()`メソッドを用いると，このように各要素が欠損値かどうかの情報を知ることができます．しかし，今の場合1個1個の値が欠損値かどうかよりも，1個以上の欠損値を含む列の情報が分かったほうが便利です（3000行以上あるデータを人の目ですべて読むのは大変です）．その場合には，`isnull()`メソッドに加え`any()`メソッドを使用します．このようにすると，各列に対し，1個以上の欠損値を含むかどうかが分かります．こちらも，下のセルで実行してみましょう．

In [ ]:
d_train_test = pd.concat([d_train, d_test], axis=0) # 訓練とテストを連結
print(d_train_test.isnull().any())

`isnull()`メソッドだけを用いたときよりも，結果が見やすいです．そして調べた結果，bmiの列に欠損値があることが分かりました．以降，このbmiの列の要素について欠損値処理を行います．ちなみにこの操作に関して，引数を追加で与えることで同様の操作を行に対して行うことも可能です．興味のある人は調べてみてください．

各列の欠損値を列ごとの（欠損していないデータの）平均値で置換する場合，`fillna()`メソッドと，`mean()`メソッドを組み合わせると簡単に行限できます．`hoge.mean()`で各列の平均値を取得でき，`hoge.fillna(fuga)`で各列の欠損値を`fuga`の対応する値で置換します．例えば，`hoge.fillna(hoge.mean())`とすると，`hoge`データセットの欠損値を各列の(欠損していないデータの)平均値で置換します．

In [ ]:
d_train_test_fill = d_train_test
d_train_test_fill['bmi'] = d_train_test_fill['bmi'].fillna(d_train_test['bmi'].mean()) # データセットの欠損値を各列の平均値で置換
n_train = len(d_train)
d_train_fill = d_train_test_fill[:n_train] # 訓練データセットの取り出し
d_test_fill = d_train_test_fill[n_train:] # テストデータセットの取り出し
print(d_train_fill)
print(d_test_fill)

もう一度，欠損値がないか確認してみましょう．すべて`False`と表示されれば，欠損値処理ができています．

In [ ]:
print(d_train_test_fill.isnull().any())

平均値でなく中央値や最頻値，指定した値での置換も同様にして行うことができます．この処理方法は，欠損値を含む列の消去（前回の方法）と比べて利点と欠点がひっくり返る形になります．つまり，利点としては欠損値を含む列にある値の情報を利用できることが挙げられ，欠点としては補完先の値を自分で設定するため利用する値によりデータセットの値が変化してしまうという点があります．どちらの手法が絶対的に優れているということはなく，データセットの値の分布を見る，あるいは複数の手法を試すなどして，どのような処理が妥当かを探っていくことが重要です．

なお，ここでは欠損値の値を補完することを紹介しましたが，「欠損値である」ことを情報の一種ととらえ，「もともと欠損値であったか」のone-hotエンコーディングを考えることもできます．興味のある人は考えてみてもいいかもしれません．

## Answer

`pd.get_dummies`で`get_dummies`というメソッドを呼び出すことができます．
最初の引数にone-hotエンコードを行う`DataFrame`を渡すので，`pd.get_dummies(d_train_test)`とすれば，とりあえずone-hotエンコードができます．
`columns_cat`に含まれる列だけone-hotエンコードをしたい場合，`columns`という名前の引数に`columns_cat`を指定することで可能になります．
したがって，`pd.get_dummies(d_train_test, columns=columns_cat)`とすればよいです．

次に，訓練とテストの取り出し方についてです．
`DataFrame`において`[i:j]`とすることで，`i`番目から`j-1`番目までの行をを取り出すことができます．
また，`[:i]`とすることで`i-1`番目まで，`[i:]`とすることで`i`番目以降の行をを取り出すことができます．
したがって，`d_train_test[:n_train]`とすることで訓練データを，`d_train_test[n_train:]`とすることでテストデータを取り出せます．

In [ ]:
d_train_test = pd.concat([d_train, d_test], axis=0) # 訓練とテストを連結
columns_cat = ["gender","ever_married", "work_type", "Residence_type", "smoking_status"] # カテゴリカル変数の列名

d_train_test_onehot = pd.get_dummies(d_train_test, columns=columns_cat) # ここを埋める．get_dummiesを使ってone-hotエンコーディング
d_train_test_onehot.pop("bmi") # 今回はbmiデータを使わず，捨てる
d_train_onehot = d_train_test_onehot[:n_train] # ここを埋める．d_train_test_onehotの訓練データ部分
d_test_onehot = d_train_test_onehot[n_train:] # ここを埋める．d_train_test_onehotのテストデータ部分
X_train_onehot = d_train_onehot.values # np.arrayに変換
X_test_onehot = d_test_onehot.values  # np.arrayに変換
